# Deprecated Here

## Clean coding

In [1]:
# Deprecated

# def liff_get_id(userid):
#     userid = str(userid)
    
    
#     liff_url = "https://%s/liffID=%s" % (server_url, userid)
#     liff_init(channel_access_token)
#     if liff_list().get("apps") != None:
#         if len(liff_list().get("apps")) >= 15:
#             for liffID in liff_list().get("apps"):
#                 liff_delete(liffID.get("liffId"))
#     liff_id = liff_add(liff_url, "compact")
    
    
#     return liff_id

In [2]:
# Deprecated
# def LIFF_get_graph_id(userid):
#     userid = str(userid)
    
#     liff_init(access_token=channel_access_token)
#     liff_url = "https://%s/graphLIFF=%s" % (server_url, userid)
#     #liff_init(channel_access_token)
#     if liff_list().get("apps") != None:
#         if len(liff_list().get("apps")) >= 15:
#             for liffID in liff_list().get("apps"):
#                 liff_delete(liffID.get("liffId"))
    
#     liff_id = liff_add(liff_url, "tall")
    
    
#     return liff_id


In [ ]:
"""
嘗試完整目前功能

"""

from kafka import KafkaProducer
from linebot.models import *
from pylineliff.liff_api import *


# Define Kafka Producer



In [4]:
producer = KafkaProducer(bootstrap_servers=["kafka:9093"],
                         key_serializer=str.encode,
                        value_serializer=lambda x : json.dumps(x).encode("utf-8"))




# Define flask 

In [5]:
"""

啟用伺服器基本樣板

"""
# 引用Web Server套件
from flask import Flask, request, abort, jsonify, render_template_string, render_template, session, redirect, url_for

# 從linebot 套件包裡引用 LineBotApi 與 WebhookHandler 類別
from linebot import (
    LineBotApi, WebhookHandler
)

# 引用無效簽章錯誤
from linebot.exceptions import (
    InvalidSignatureError
)

# 載入json處理套件
import json

# 先取得基本資料
secretFileContentJson=json.load(open("./line_secret_key",'r'))
channel_access_token = secretFileContentJson["channel_access_token"]
secret_key = secretFileContentJson["secret_key"]
self_user_id = secretFileContentJson["self_user_id"]
rich_menu_id = secretFileContentJson["rich_menu_id"]
server_url = secretFileContentJson["server_url"]

# 設定Server啟用細節
app = Flask(__name__,static_folder = "./Images",static_url_path = "/Images")

# 生成實體物件
# line-bot
# 給予 one paramater channel_access_token
# 給予 公司章 讓 line_bot_api 送到 LINE時 能被認證為我們的人員
line_bot_api = LineBotApi(channel_access_token)

# 生成 handler
# 要給予認證章 以便 handler 辨識出給我們的訊息
handler = WebhookHandler(secret_key)

# 啟動server對外接口，使Line能丟消息進來
# LINE 都是透過 POST 來發送訊息
@app.route("/", methods=["POST"])

def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']
    
    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body : " + body)
    
    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)

    return 'OK'




In [6]:
"""
用戶follow 就給目前的菜單

"""
# 準備 greeting 訊息
greeting_message_list = [
    TextSendMessage(
        text = "歡迎來到想吃啥"
    ),
    TextSendMessage(
        text = "請使用下方功能選單"        
    ),
    TextSendMessage(
        text = "任何問題，請使用 [聯絡我們]"
    )
    
    
]




# 載入Follow事件
from linebot.models import FollowEvent

# 載入requests套件
import requests

# 告知handler，如果收到FollowEvent，則做下面的方法處理
@handler.add(FollowEvent)
def reply_text_and_get_user_profile(event):
    
    # 先取出用戶資料
    user_id = line_bot_api.get_profile(event.source.user_id)
    
    # 將用戶資訊存在檔案內
    with open("./users.txt", "a+") as myfile:
        myfile.write(json.dumps(vars(user_id),sort_keys=True))
        myfile.write("\r\n")
        
    # 將 菜單綁至用戶上
    linkRichMenuId= rich_menu_id
    linkMenuEndpoint='https://api.line.me/v2/bot/user/%s/richmenu/%s' % (event.source.user_id, linkRichMenuId)
    linkMenuRequestHeader={'Content-Type':'image/jpeg','Authorization':'Bearer %s' % channel_access_token}
    lineLinkMenuResponse=requests.post(linkMenuEndpoint,headers=linkMenuRequestHeader)
    
    # 回覆文字消息與圖片消息
    line_bot_api.reply_message(
        event.reply_token,
        greeting_message_list
    )












# Global function

## 產生 LIFF ID

In [7]:
def apply_liff_id(userid, route, pattern):
    if type(pattern) != str:
        raise ValueError('Pattern can only be string e.g: (compact, tall, full)')
    elif type(route) != str:
        raise ValueError('Route can only be string without / e.g: (compact, tall, full)')
    elif route.find("/") != -1:
        raise ValueError('Route can only be string without / e.g: (compact, tall, full)')
        
    # make sure userid is string
    userid = str(userid)
    liff_init(access_token=channel_access_token)
    liff_url = "https://%s/%s%s" % (server_url, route, userid)
    
    # make sure liff amount not exceed 15
    if liff_list().get("apps") != None:
        if len(liff_list().get("apps")) >= 15:
            for liffID in liff_list().get("apps"):
                liff_delete(liffID.get("liffId"))
    
    liff_app_id = liff_add(liff_url, pattern)
    
    return liff_app_id
    

# 針對吃啥的回應

## 這邊要放入主要推薦系統

In [8]:
"""
針對 [::text:] 吃啥 的回應


"""
from mysql_account import MyAccount
import MySQLdb
from rate import *
import time


def flex_send_whateat():
    a = MyAccount()
    # charset = utf-8
    conn = MySQLdb.connect(a.host, a.account, a.passwd, db="test", charset="utf8")
    start = time.time()
    cursor = conn.cursor()
    days = get_days()

    cursor.execute("SELECT Name,rate,address,%s,phone from res where rate >= 3 && %s != '' order by rand() limit 4" % (days,days))
    tu = cursor.fetchall()
    conn.close()
    
    # 對 Tuple 內所有參數 使用 gen_bubble 產生 bubble_message
    # 用 map 搭配 lambda 
    # 再用 list 包住 傳給 gen_flex 產生 flexMessage
    bubble_list = list(map(lambda x: gen_bubble(x),tu))
    
    bubble_content = gen_flex(bubble_list)
    flex_content = CarouselContainer.new_from_json_dict(json.loads(bubble_content))
    
    Flex_message = FlexSendMessage(alt_text="Flex", contents=flex_content)
    print("=====Time Spending=====")
    print(time.time() - start)
    return Flex_message







# 針對圖片推薦的回應

## 這邊預計要放入 html 來點選圖片


流程 

先回覆flex 點選兩個圖片選一

回傳後再次送兩個選一

重複上述

算出推薦後回覆

In [9]:

# 塞進 flex bubble
def graph_liff_message(userID):
    User = userID
    liffID = apply_liff_id(User, route='graphLIFF=', pattern='tall')
    graph_liff_url = "line://app/%s" % liffID.get("liffId")
    
    liff_flex_bubble = graph_bubble(graph_liff_url)

    bubble_container = BubbleContainer.new_from_json_dict(json.loads(liff_flex_bubble))

    liff_message = FlexSendMessage(alt_text="Check on your phone",contents=bubble_container)
    return liff_message

In [10]:
"""
針對 [::text:] 圖片推薦 的回應


"""


"""
[::text:] 圖片推薦 get_img_reply_flex()
"""


from images_reply_version2 import get_image_reply_html
@app.route("/graphLIFF=<userid>")

def graph(userid):
    html = get_image_reply_html(userid)
    
    
    
    return render_template_string(html)




In [11]:
@app.route("/post_graph", methods=["POST"])
def post_graph():
        
    body = json.loads(request.get_data(as_text=True))
    
    # {'graph': {'美食': 0, '影音藝文': 0, '3C商品': 0, '休閒旅遊': 0, '美容美妝': 0, '生活服務': 0}, 'user': 'us'}
    user = body.get("user")
    graph = body.get("graph")
    
    # graph = {'美食': 0, '影音藝文': 0, '3C商品': 0, '休閒旅遊': 0, '美容美妝': 0, '生活服務': 0}
    
    
    graph_future = producer.send(topic="mytopics", key=user, value=list(graph.values()))
    graph_future.get(timeout=50)
    print(body)
    
        
    line_bot_api.push_message(
        body.get("user",None),
        TextSendMessage(
            text = "Your select graph data is get by lineBot ! Please Wait"
        )
    )
    
    
   
    return "OK"

# 針對位置推薦


### 流程 :

用戶點了位置推薦 -> 取得用戶ＩＤ 並裝進 HTML LIFF -> 用戶點了回傳位置後 -> 傳回 post back location 接口 並傳回告知可關閉資訊


-> 接口取得用戶ＩＤ與位置資訊後 分析位置模式 判斷在哪個公車上 -> 推薦資訊透過推播訊息完成

In [12]:
"""
Testing 5second send location

"""
from location_reply import *
@app.route("/liffID=<userid>")
def liffID(userid):
    html = location_html(userid=userid)
    
    
    
    return render_template_string(html)
    
    

    


In [13]:
def location_liff_message(userID):
    user = userID
    
    liffID = apply_liff_id(user, route='liffID=', pattern='compact')    
    location_liff_url = "line://app/%s" % liffID.get("liffId")
    
    liff_flex_bubble = location_bubble(location_liff_url)
    
    bubble_container = BubbleContainer.new_from_json_dict(json.loads(liff_flex_bubble))
    liff_message = FlexSendMessage(alt_text="Check on your phone",contents=bubble_container)
    
    return liff_message

# 處理丟過來的 位置資訊

## 算出週圍多邊形後

## 丟入 sql st_within

## push 推薦內容

In [14]:
@app.route("/post_location", methods=["POST"])
def post_location():
    
    
    body = json.loads(request.get_data(as_text=True))
    
    
    
    with open("./location.txt", "a+") as f:
        f.write(json.dumps(body))
        
        f.write("\r\n")
    line_bot_api.push_message(
        body.get("user",None),
        TextSendMessage(
            text = "Your data is get by lineBot ! Please Wait"
        )
    )
    
    location = body.get("location")
    sql_location_string = ""
    if location:

        lat = location[2].get("lat")
        lon = location[2].get("lng")
        tups = count_GPS(lat,lon)
    
        new_tuple = ()
        for tup in tups:        
            a,b = tup
            req_string = ""
            req_string += str(a) + " " + str(b)
            new_tuple += (req_string,)
    
        sql_location_string = ",".join(new_tuple)
        
    else:
        # android 無法使用 LIFF geolocation
        line_bot_api.push_message(
        body.get("user",None),
        [TextSendMessage(
            text = "抱歉 !請以Chrome開啟網頁並重試"
        ),
        TextSendMessage(
            text = "https://%s/liffID=%s" % (server_url,body.get("user"))
        )]
        )
        return "OK"
    
    # connect to sql
    a = MyAccount()
    # charset = utf-8
    conn = MySQLdb.connect(a.host, a.account, a.passwd, db="test", charset="utf8")
    start = time.time()
    cursor = conn.cursor()
    days = get_days()
    tu = ()
    if sql_location_string:
        sql_string = """SELECT Name,rate,address,%s,phone 
            from res 
            where (select ST_Within(
                res.geo,
                ST_GeomFromText(
                    'POLYGON((%s))',4326)
                ) = True
                 ) && rate >= 3 && %s != '' 
            order by rand() 
            limit 5""" % (days,sql_location_string,days)

        cursor.execute(sql_string)
        tu = cursor.fetchall()
    if len(tu) == 0:
        line_bot_api.push_message(
        body.get("user",None),
        TextSendMessage(
            text = "Sorry ! There is no restaurant near your location"
            )
        )
        
    else:    
        # 對 Tuple 內所有參數 使用 gen_bubble 產生 bubble_message
        # 用 map 搭配 lambda 
        # 再用 list 包住 傳給 gen_flex 產生 flexMessage
        bubble_list = list(map(lambda x : gen_bubble(x),tu))
        flex = gen_flex(bubble_list)
        carousel_content = CarouselContainer.new_from_json_dict(json.loads(flex))
        message = FlexSendMessage(alt_text="Check phone",contents=carousel_content)

        line_bot_api.push_message(
            body.get('user',None),
            message
        )
    print("============tuple===========")
    print(tu)
        
        
    return "OK"

# 針對聯絡我們的回應

## 這邊要放入主要部份回應 quick button

In [15]:
"""
針對 [::text:] 聯絡我們 的回應


"""

# 創建quick button


# 創建文字快捷
textQuickButton = QuickReplyButton(
    action=MessageAction(
        label="文字消息",
        text="發送這個消息"
    )

)

# 創建選擇日期快捷
dataQuickButton = QuickReplyButton(
    action=DatetimePickerAction(
        label="選擇日期",
        data="data3",
        mode="date",
        initial="2013-04-01",
        min="2011-06-23",
        max="2019-04-23"
    )
)

# 創建相機使用快捷
cameraQuickButton = QuickReplyButton(
    action=CameraAction(
        label="拍個照！開心開心"
    )
)

# 相機卷使用快捷
cameraRollQuickButton = QuickReplyButton(
    action=CameraRollAction(
        label="選擇相片！爽爽爽"
    )
)

# 傳送地點快捷
locationQuickButton = QuickReplyButton(
    action=LocationAction(
        label="上傳地點！ㄏ ㄏ "
    )
)

# PostBack 快捷
postBackQuickButton = QuickReplyButton(
    action=PostbackAction(
        label="我是postback事件",
        display_text='postback text2',
        data='action=buy&itemid=2'
        
    )
)

# 創建 quick reply button List
quickReplyList = QuickReply(
    items=[textQuickButton, dataQuickButton, cameraQuickButton, cameraRollQuickButton, locationQuickButton, postBackQuickButton]
)

# 封裝進 textsendMessage
quickReplyTextSendMessage = TextSendMessage(text="發送你的問題！",quick_reply=quickReplyList)




# 定義Main Page

## flask get return html

### 傳入餐廳 ID UserID

In [ ]:
from mysql_account import MongoBase
import pymongo
from MainPage import res_html
@app.route("/index/resid=", methods=["GET"])
def main_page():
    m = MongoBase()
    mongo = pymongo.MongoClient("mongodb://mongodb:27017/",
                                username=m.username,
                                password=m.password,
                                authSource=m.authSource,
                                authMechanism=m.authMechanism
                                )
    db = mongo["res"]
    col = db["resInfo"]
#     query = {"phone": "0221819999"}

#     doc = col.find(query, {"_id": 0, "comment": 0, "content": 0})
    doc = col.find({}, {"_id": 0, "comment": 0, "content": 0})
    
    return render_template_string(res_html(doc))

    
    



# 定義 協同式過濾

## 開啟接口給 MainPage ajax request

In [ ]:
@app.route("/countRes", methods=["POST"])
def countRes():
    data = json.loads(request.get_data(as_text=True))
    
    resID = data.get("resID")
    
    response_data = {
        "resID": resID,
        "String": "Hello"
    }
    
    future = producer.send(topic="mytopics", key="test", value=response_data)
    future.get(timeout=50)
    return redirect("https://91e47741.ngrok.io/test")
    


In [ ]:
@app.route("/test", methods=["GET"])
def testing():
    from kafka import KafkaConsumer
    consumer = KafkaConsumer("mytopics", bootstrap_servers=["kafka:9093"], auto_offset_reset='latest') 
    response_data=[]
    
#     for msg in consumer:
        
#         value = json.loads(msg.value)
        
#         response_data.append(value)
#         print(response_data[-1])
#         return jsonify(response_data[-1])
    return "ok"
    
    

# Handler define here

### 遇到的問題 : 

* handler觸發後 回應準備字典內的所有方法都觸發

* 預期為 : 對應哪個key 只觸發指定的 value

* 修正 : 定義傳遞方法

* 另個修正 : 定義空字典 handler 觸發後 重新修改 指定 value 或 方法 

In [ ]:
import random
import re

text_reply_dict = {
    1: TextSendMessage(text="醒醒 !"),
    2: TextSendMessage(text="醒 !"),
    3: TextSendMessage(text="你沒有 !"),
    4: TextSendMessage(text="該起床了 !"),
    5: TextSendMessage(text="醒了沒 ?"),
}
#text_reply_dict.get(random.randint(1,4))
#random.randint(1,3)

In [ ]:
def parseDict(key,userid=None):
    if key == "[::text:] 位置推薦":
        return location_liff_message(userid)
    elif key == "[::text:] 吃啥":
        return flex_send_whateat()
    elif key == "[::text:] 圖片推薦":
        return graph_liff_message(userid)
    elif key == "[::text:] 聯絡我們":
        return quickReplyTextSendMessage
    
    
    
    
    
    
    

In [ ]:
"""
Handler 處理進入的所有消息


"""
@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    
    
    if(event.message.text.find('::text:')!= -1):
        
        
        
        line_bot_api.reply_message(
        event.reply_token,
        parseDict(key=event.message.text,userid=event.source.user_id))
    elif re.findall(r"妹+|婆+|[女友]+|[女朋友]+",event.message.text) :
        line_bot_api.reply_message(
        event.reply_token,
        text_reply_dict.get(random.randint(1,5)))
    
    else:
        line_bot_api.reply_message(
        event.reply_token,
        TextSendMessage(text="請點擊菜單 來取得更多資訊喔"))

In [ ]:
"""
Handler postback 處理

"""

@handler.add(PostbackEvent)
def handle_post_message(event):
    user_profile = line_bot_api.get_profile(event.source.user_id)
    if event.postback.data.find("貓") == 0:
        print(event.postback)
    else:
        print("WTF")



In [ ]:
if __name__ == "__main__":
    
    app.run(host="0.0.0.0", threaded=True)

 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)
172.25.0.6 - - [18/Mar/2019 06:08:30] "GET /graphLIFF=U106173a60887e3262a4fa2ceeacaed87 HTTP/1.1" 200 -
172.25.0.6 - - [18/Mar/2019 06:08:34] "POST /post_graph HTTP/1.1" 200 -


{'graph': {'美食': 0, '影音藝文': 0, '3C商品': 0, '休閒旅遊': 0, '美容美妝': 0, '生活服務': 0, 'Paris': 1}, 'user': 'U106173a60887e3262a4fa2ceeacaed87'}
